In [1]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import os
import json
from tools.data_utils import *
from tools.optimization import EarlyStoppingCallback, evaluate_model
import matplotlib.pyplot as plt
import argparse
import time
from tools.NPmodels import *
from tools.NPtrain import process_batch, NeuralProcessTrainer, evaluate_np
from tabulate import tabulate
import torch.optim.lr_scheduler as lr_scheduler
import time
from torch.utils.data import Subset

Customized collate function for pin problems

It outputs three items: images, pins (coordinates), and labels

In [2]:
def custom_collate_fn(batch):
    images = [sample['image'] for sample in batch]
    pins = [sample['pins'] for sample in batch]
    outputs = [sample['outputs'] for sample in batch]

    return {
        'image': torch.stack(images, dim=0),
        'pins': pins,
        'outputs': outputs}

In [3]:
def data_prepare(config, seed):
    dataset = config['dataset']
    mesh = True if config['mode'] == "mesh" else False
    feature_extracted = True if config['feature'] == "DDPM" else False
    modality = config['modality']
    batch_size = config['batch_size']
    n_pins = config['n_pins']
    d = config['d']
    r = config['r']

    if dataset == "Synthetic":
        input_shape = 28
        if feature_extracted:
            input_channel = 74
        else:
            input_channel = 3
    elif dataset == "PinMNIST":
        input_shape = 28
        if feature_extracted:
            input_channel = 71
        else:
            input_channel = 1
    elif dataset == "Building":
        input_shape = 100
        if feature_extracted:
            input_channel = 3584
        else:
            if modality == "PS-RGBNIR":
                input_channel = 4
            elif modality == "PS-RGB":
                input_channel = 3
            elif modality == "PS-RGBNIR-SAR":
                input_channel = 8
    elif dataset == "Cars":
        if feature_extracted:
            # TO DO: Check how many features does the DDPM version has
            print('DDPM is still not available for this dataset')
        else:
            input_channel = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if feature_extracted:
        folder = f"{dataset}_ddpm"
    else:
        folder = f"{dataset}"

    if dataset == "PinMNIST":
        test_data_folder = f"./data/{folder}/random_fixedTrue_{n_pins}pins_{28}by{28}pixels_{r}radius_4seed"
        if mesh:
            data_folder = f"./data/{folder}/mesh_{d}step_{28}by{28}pixels_{r}radius_4seed"
            config['n_pins'] = (28 // d + 1) ** 2
        else: # Random pins 
            data_folder = test_data_folder

    elif dataset == "Synthetic":
        folder += "/28by28pixels_1000images_123456seed"
        test_data_folder = f"./data/{folder}/random_{n_pins}pins"
        if mesh:
            data_folder = f"./data/{folder}/mesh_{d}step_pins"
            config['n_pins'] = (28 // d + 1) ** 2
        else:

            data_folder = test_data_folder
    elif dataset == "Building":
        test_data_folder = f"./data/{folder}/random_n_pins_{n_pins}"
        if mesh:
            data_folder = f"./data/{folder}/mesh_{d}_step"
            config['n_pins'] = (100 // d + 1) ** 2
        else:
            data_folder = f"./data/{folder}/random_n_pins_{n_pins}"
            data_folder = test_data_folder
            
    elif dataset == "Cars":
        r = 100
        test_data_folder = f"./data/{folder}/test/random_fixedTrue_{n_pins}pins_{800}by{800}pixels_{r}radius_{seed}seed"
        if mesh:
            train_data_folder = f"./data/{folder}/train/mesh_{d}step_{800}by{800}pixels_{r}radius_{seed}seed"
            val_data_folder = f"./data/{folder}/val/mesh_{d}step_{800}by{800}pixels_{r}radius_{seed}seed"
            config['n_pins'] = (800 // d + 1) ** 2
        else: # Random pins 
            data_folder = test_data_folder

    if dataset == "Building":
        transform = transforms.Compose([
        ToTensor(),  # Convert to tensor (as you were doing)
        Resize100(),  # Resize to 100x100
    ])
    elif dataset == "Cars":
        transform = transforms.Compose([
        ExtractImage(), # Get image from image and mask combination
        ToTensor(),  # Convert to tensor (as you were doing)
        Resize200(),  # Resize to 200x200
    ])
    else:
        transform = transforms.Compose([
        ToTensor(),  # Convert to tensor (as you were doing)
        Resize()  # Resize to 100x100
    ])        
    # As DDPM does not work well with Rotterdam Building dataset, we have not explored this dataset with different modalities with DDPM
    if dataset == "Building":
        root_dir = f"./data/Building/{modality}/"
        transformed_dataset = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                     root_dir=root_dir, modality=modality,
                                     transform=transform)
        test_dataset = PinDataset(csv_file=f"{test_data_folder}/pins.csv",
                                     root_dir=root_dir, modality=modality,
                                     transform=transform)
    elif dataset == "Cars":
        root_dir=f"./data/{folder}/images/"
        train_dataset = PinDataset(csv_file=f"{train_data_folder}/pins.csv",
                                     root_dir=root_dir, transform=transform)
        val_dataset = PinDataset(csv_file=f"{val_data_folder}/pins.csv",
                                     root_dir=root_dir, transform=transform)
        eval_dataset = PinDataset(csv_file=f"{test_data_folder}/pins.csv",
                                     root_dir=root_dir, transform=transform)
    else:
        root_dir=f"./data/{folder}/images/"
        transformed_dataset = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                     root_dir=root_dir, transform=transform)
        test_dataset = PinDataset(csv_file=f"{test_data_folder}/pins.csv",
                                     root_dir=root_dir, transform=transform)

    dataset_size = len(transformed_dataset)
    train_size = int(0.7 * dataset_size)
    val_size = int(0.10 * dataset_size)
    test_size = dataset_size - train_size - val_size

    if dataset != "Cars":
        if os.path.exists(f"./data/{dataset}/train_indices.npy"):
            train_indices = np.load(f'./data/{dataset}/train_indices.npy')
            val_indices = np.load(f'./data/{dataset}/val_indices.npy')
            test_indices = np.load(f'./data/{dataset}/test_indices.npy')
            # Use the indices to create new datasets
            train_dataset = Subset(transformed_dataset, train_indices)
            val_dataset = Subset(transformed_dataset, val_indices)
            # test_dataset = Subset(transformed_dataset, test_indices)
             # Use the indices to create new test datasets
            eval_dataset = Subset(test_dataset, test_indices)
        else:
            # Split the dataset into train, validation, and test sets
            train_dataset, val_dataset, test_dataset = random_split(
                transformed_dataset, [train_size, val_size, test_size]
            )
            np.save(f'./data/{dataset}/train_indices.npy', train_dataset.indices)
            np.save(f'./data/{dataset}/val_indices.npy', val_dataset.indices)
            np.save(f'./data/{dataset}/test_indices.npy', test_dataset.indices)
            # Use the indices to create new test datasets
            eval_dataset = Subset(test_dataset, test_dataset.indices)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    return input_channel, input_shape, train_loader, val_loader, eval_loader

Experiment pipeline for NP

In [4]:
def run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, epochs, val_every_epoch, config, device, num_runs=3, print_freq=2):
    test_losses = []
    # partial_percent = config['partial_percent']
    experiment_id = int(time.time())
    best_val_loss_NP = float('inf')
    # config['experiment_id'] = experiment_id
    experiment_id = config['experiment_id']

    r_dim = np_config.r_dim
    h_dim = np_config.h_dim
    z_dim = np_config.z_dim
    lr = config['best_lr']

    # Create storage directory and store the experiment configuration
    if not os.path.exists(f'./history/neural_processes/{experiment_id}'):
        os.makedirs(f'./history/neural_processes/{experiment_id}')
    with open(f"./history/neural_processes/{experiment_id}/config_np.json", "w") as outfile: 
        json.dump(config, outfile)
        
    global_val_loss = float('inf')
    
    for run in range(num_runs):
        count = 0
        GP_test_losses = []
        
        early_stopping = EarlyStoppingCallback(patience=5, min_delta=0.001)
        model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        np_trainer = NeuralProcessTrainer(device, model, optimizer, early_stopping, experiment_id, print_freq=print_freq)          

        np_trainer.train(train_loader, val_loader, epochs)
        if np_trainer.best_val_loss <= global_val_loss:
            global_val_loss = np_trainer.best_val_loss 
            torch.save(np_trainer.neural_process.state_dict(), f'./history/neural_processes/{experiment_id}' + f'/best_{args.dataset}_np.pt')
        count += 1

    return experiment_id


# Function to run the pipeline and save data
def run_and_save_pipeline_np(train_loader, val_loader, test_loader, epochs, val_every_epoch, config, num_runs, device):
    test_partial_percents = [0.25, 0.5, 0.75, 1]
    test_losses = []
    r2_list = []
    table = []
    table.append(['Dataset', 'Mode', 'd', 'n_pins', 'LR', 'PLP', 'MSE error', 'R2'])
    experiment_id = run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, epochs, val_every_epoch, config, device, num_runs)
    # Run final testing
    model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
    # MSE
    model.load_state_dict(torch.load(f'./history/neural_processes/{experiment_id}/best_{args.dataset}_np.pt'))
    for partial_percent in test_partial_percents:
        test_loss, r2 = evaluate_np(model, test_loader, device, partial_percent=partial_percent)
        print(f"pp: {partial_percent} MSE loss: {test_loss} R2 score: {r2}")
        table.append([args.dataset, args.mode, args.d, args.n_pins, config['best_lr'], partial_percent, test_loss, r2])
        test_losses.append(test_loss)
        r2_list.append(r2)
    table = tabulate(table, headers='firstrow', tablefmt='fancy_grid', showindex=True)
    print(table)
    with open('./history/neural_processes/table.txt', 'a') as f:
        f.write('\n')
        f.write(table + '\n')  # Add a newline character after writing the table
        f.write('\n')  # Add an additional newline character for separation
    print("saved")
    return test_losses, r2_list, experiment_id

Configs

In [5]:
class Args:
    def __init__(self):
        self.dataset = "Synthetic"
        self.n = 1000
        self.mode = "random"
        self.feature = "AE"
        self.modality = "PS-RGBNIR"
        self.d = 10
        self.n_pins = 100
        self.r = 3
        self.partial_percent = 0.8
        self.epochs = 1000
        self.batch_size = 50
        self.learning_rate = 1e-4
        self.val_every_epoch = 10
        self.num_runs = 1
        self.seed = 4

class NP_config:
    def __init__(self):
        self.r_dim = 512
        self.h_dim = 512
        self.z_dim = 512
        self.lr = 4e-5
        self.epochs = 100

np_config = NP_config()
args = Args()

In [6]:
# Set a random seed for PyTorch
seed = 4  # You can use any integer value as the seed
torch.manual_seed(seed)
# Set a random seed for NumPy (if you're using NumPy operations)
np.random.seed(seed)

 # Choose datasets
dataset = args.dataset 
n = args.n
mode = args.mode
d = args.d
n_pins = args.n_pins
r = args.r
partial_percent = args.partial_percent

# Set your hyperparameters
epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.learning_rate
val_every_epoch = args.val_every_epoch
num_runs = args.num_runs
   
# config = {}
config = vars(args)
config["experiment_id"] = 0
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading datasets

In [7]:
input_channel, input_shape, train_loader, val_loader, eval_loader = data_prepare(config, seed)

Train neural processes with custom learning rates and output the summary table

In [ ]:
start_time = time.time()
# define the NP 
r_dim, z_dim, h_dim = np_config.r_dim, np_config.z_dim, np_config.h_dim
neural_processes = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
optimizer = torch.optim.Adam(neural_processes.parameters(), learning_rate)
config['best_lr'] = 1e-3
# Run and save the pipeline data
test_losses, r2, experiment_id = run_and_save_pipeline_np(train_loader, val_loader, eval_loader,\
                                           epochs, val_every_epoch, config, num_runs, device)
end_time = time.time()
elapsed_time = end_time - start_time
print("Time elapsed:", elapsed_time, "seconds")

Epoch: 0, Avg_loss: 1425.687
Epoch: 0, Val_loss 556.013
Epoch: 1, Avg_loss: 395.175
Epoch: 2, Avg_loss: 364.217
Epoch: 2, Val_loss 363.040
Epoch: 3, Avg_loss: 361.702
Epoch: 4, Avg_loss: 360.983
Epoch: 4, Val_loss 360.914
Epoch: 5, Avg_loss: 360.404
Epoch: 6, Avg_loss: 360.191
Epoch: 6, Val_loss 360.526
Epoch: 7, Avg_loss: 359.719
Epoch: 8, Avg_loss: 359.144
Epoch: 8, Val_loss 357.113
